In [46]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import requests
import statistics
import csv

# Assign API URL to a variable
mars_url = "https://api.maas2.apollorion.com"

# Initialize lists to use later for statistics for 3 json keys
# def get_sols_from_file():
#     """
#     Designed to open a csv file called sols.csv which will contain
#     a range of comma separated sols. For example, the file I've included contains 
#     the last 60 days worth of data. Just for an FYI the latest available sol we
#     can query is number 2667. The function reads the file, stores and returns the 
#     values as a list.
#     """
#     with open('sols.csv', newline='') as sols:
#         query = list(csv.reader(sols))
#     return query

json_list = []

# intiating a call to request all available data from the API

for sol in range(0,2667):
    data = requests.get(mars_url + "/" + str(sol) + "/")
    
    # Concatenated lines because I didn't need the 'json_obj' variable anymore
    # since the list is what is going to store all the data and wanted to
    # attempt chaining.
    
    json_list.append(json.loads(data.text))
    
    """
    This part took me a long time to figure out since it's not intuitive at 
    first glance. The json object the API returns is just one object which 
    can't be loaded into the DataFrame.from_dict() method with automatic 
    recognition because it's a scalar (single) value. The only way it can be 
    converted to a dataframe with this method is if an index is given, however
    this means that the header row is transposed to be row headers, which won't
    work for writing to a csv and further pandas manipulation.
    What I figured out was that I needed to have whatever object was passed to
    the from_dict() method would be multiple values, so the dictionary's key
    values could become the column headers. I solved this by initiating and
    empty list and appending all queried json objects from the API to the list.
    The from_dict() method then is able to automatically convert the list of 
    json objects (dictionary type) to a dataframe.
    """
dataframe = pd.DataFrame.from_dict(json_list) 


print(dataframe.head())




KeyboardInterrupt: 

In [45]:
"""
Write the dataframe to a csv file to be used later
Purpose is to not have to query the API constantly for all this data

Variables explained:
    # Using absolute path to write to the same directory containing Jupyter Notebook
    # Don't need an index so the variable is false
    # Representing missing data as 'NaN' since pandas has built in tools to handle these
    # Setting any floats to only have 2 points after the decimal
"""

dataframe.to_csv('/Users/zack/dat129_ccac/Homework/final_project/martian_weather.csv', 
                 index=False, 
                 na_rep='NaN', 
                 float_format='%.2f')

In [47]:
# Load dataframe from the csv to be manipulated and edited
weather_df = pd.read_csv('martian_weather.csv')

#
weather_df.head()

,status,message,id,terrestrial_date,ls,season,min_temp,max_temp,pressure,pressure_string,...,atmo_opacity,sunrise,sunset,local_uv_irradiance_index,min_gts_temp,max_gts_temp,wind_direction,sol,unitOfMeasure,TZ_Data
0,404,Sol not found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200,NaN,1.0,2012-08-07T00:00:00.000Z,150.0,Month 6,NaN,NaN,NaN,Lower,...,Sunny,05:30,17:22,NaN,NaN,NaN,NaN,1.0,Celsius,America/Port_of_Spain
2,404,Sol not found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,404,Sol not found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,404,Sol not found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#     max_temp.append(json_obj['max_temp'])
#     min_temp.append(json_obj['min_temp'])
#     atmospheric_opacity.append(json_obj['atmo_opacity'])

#     temp_max_mean = statistics.mean(max_temp)
#     max_max_temp = max(max_temp)
#     temp_min_mean = statistics.mean(min_temp)
#     min_min_temp = min(min_temp)
#     atmo_mode = statistics.mode(atmospheric_opacity)

    # Now we're going to write out the report in a text file for the user
    # Open a new file and create some strings to contextualize the output
#     with open('martian_weather_report.txt', 'w') as report:
#         line_one = "The amount of sols you queried from the Curiosity data was: " 
#         line_two = "The average high temperature (in Celsius): "
#         line_three = "The average low temperature (in Celsius): "
#         line_four = "High temperature extreme (in Celsius): "
#         line_five = "Low temperature extreme (in Celsius): "
#         line_six = "Most common atmospheric conditions: "
    
#     #Write the above strings and their corresponding values to the report 
#         report.write(line_one)
#         report.write(str(len(query_list[0])))
#         report.write("\n")
#         report.write(line_two)
#         report.write(str(temp_max_mean))
#         report.write("\n")
#         report.write(line_three)
#         report.write(str(temp_min_mean))
#         report.write("\n")
#         report.write(line_four)
#         report.write(str(max_max_temp))
#         report.write("\n")
#         report.write(line_five)
#         report.write(str(min_min_temp))
#         report.write("\n")
#         report.write(line_six)
#         report.write(atmo_mode)

# def main():
#     #sol_range = get_sols_from_file()
#     print()
#     print("Please be patient, this may take some time. Pretend the Jeorpardy song is playing.")
#     print()
#     parse_martian_json_data(sol_range)
#     print()
#     print("Your report is now complete.")
#     print("Please open 'martian_weather_report.txt' to view your results.")
#     print()
#     print("Over the final 60 sols worth of weather data recorded by the Curiosity rover" +
#         " we can determine the following statistics about the temperature and" +
#         " and atmospheric conditions on Mars.")
#     print("")
#     print("The average high temperature: {:4,.1f} degrees Celsius.".format(temp_max_mean))
#     print("The average low temperature: {:4,.1f} degrees Celsius.".format(temp_min_mean))
#     print("Atmospheric extremes reached lows of", min_min_temp, " degrees Celsius and highs of", max_max_temp, "degrees Celsius.")
#     print()
#     print("For what it's worth, most of the days it was", atmo_mode, ".")


# def main():
#     parse_martian_json_data()

# if __name__ == "__main__":
#     main()  